In [7]:
#Installation de la librairie pandas-profiling (à faire une fois), redémarrer le kernel jupyter ensuite

In [ ]:
pip install pandas-profiling

In [47]:
#Installation de la librairie researchpy (à faire une fois), redémarrer le kernel jupyter ensuite

In [ ]:
pip install researchpy

In [1]:
#Chargement des librairie nécessaires
import os
import argparse
import pandas as pd
import numpy as np
import statsmodels.api
from sklearn import preprocessing
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport

In [2]:
chemin=os.getcwd()
df = pd.read_csv(chemin + "/" + "data_v1.0.csv", encoding="unicode_escape")
df.head()

,Unnamed: 0,index,date,cheveux,age,exp,salaire,sexe,diplome,specialite,note,dispo,embauche
0,0,0,2012-06-02,roux,25.0,9.0,26803.0,F,licence,geologie,97.08,non,0
1,1,1,2011-04-21,blond,35.0,13.0,38166.0,M,licence,forage,63.86,non,0
2,2,2,2012-09-07,blond,29.0,13.0,35207.0,M,licence,geologie,78.50,non,0
3,3,3,2011-07-01,brun,NaN,12.0,32442.0,M,licence,geologie,45.09,non,0
4,4,4,2012-08-07,roux,35.0,6.0,28533.0,F,licence,detective,81.91,non,0


In [5]:
df.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  20000 non-null  int64  
 1   index       20000 non-null  int64  
 2   date        19909 non-null  object 
 3   cheveux     19897 non-null  object 
 4   age         19909 non-null  float64
 5   exp         19904 non-null  float64
 6   salaire     19905 non-null  float64
 7   sexe        19900 non-null  object 
 8   diplome     19890 non-null  object 
 9   specialite  19907 non-null  object 
 10  note        19886 non-null  float64
 11  dispo       19894 non-null  object 
 12  embauche    20000 non-null  int64  
dtypes: float64(4), int64(3), object(6)
memory usage: 2.0+ MB


In [6]:
df.describe ()

,Unnamed: 0,index,age,exp,salaire,note,embauche
count,20000.000000,20000.000000,19909.000000,19904.000000,19905.000000,19886.000000,20000.000000
mean,9999.500000,9999.500000,35.004521,9.500352,34974.343934,75.168762,0.114600
std,5773.647028,5773.647028,9.609169,3.012109,5003.099616,17.095926,0.318547
min,0.000000,0.000000,-3.000000,-2.000000,14128.000000,8.680000,0.000000
25%,4999.750000,4999.750000,29.000000,7.000000,31596.000000,63.720000,0.000000
50%,9999.500000,9999.500000,35.000000,9.000000,34979.000000,75.080000,0.000000
75%,14999.250000,14999.250000,41.000000,12.000000,38358.000000,86.617500,0.000000
max,19999.000000,19999.000000,74.000000,23.000000,53977.000000,143.220000,1.000000


#### Chargement d'un nouveau dataframe avec uniquement les colonne 2 à 12 incluses

In [7]:
chemin=os.getcwd()
df = pd.read_csv(chemin + "/" + "data_v1.0.csv", index_col= 0, header=0, usecols=range(2,13),encoding="unicode_escape")
df.head()

,cheveux,age,exp,salaire,sexe,diplome,specialite,note,dispo,embauche
date,,,,,,,,,,
2012-06-02,roux,25.0,9.0,26803.0,F,licence,geologie,97.08,non,0
2011-04-21,blond,35.0,13.0,38166.0,M,licence,forage,63.86,non,0
2012-09-07,blond,29.0,13.0,35207.0,M,licence,geologie,78.50,non,0
2011-07-01,brun,NaN,12.0,32442.0,M,licence,geologie,45.09,non,0
2012-08-07,roux,35.0,6.0,28533.0,F,licence,detective,81.91,non,0


In [8]:
df.describe ()

,age,exp,salaire,note,embauche
count,19909.000000,19904.000000,19905.000000,19886.000000,20000.000000
mean,35.004521,9.500352,34974.343934,75.168762,0.114600
std,9.609169,3.012109,5003.099616,17.095926,0.318547
min,-3.000000,-2.000000,14128.000000,8.680000,0.000000
25%,29.000000,7.000000,31596.000000,63.720000,0.000000
50%,35.000000,9.000000,34979.000000,75.080000,0.000000
75%,41.000000,12.000000,38358.000000,86.617500,0.000000
max,74.000000,23.000000,53977.000000,143.220000,1.000000


In [14]:
print ("#[1]============ Profilage du jeux de données ================")
from pandas_profiling import ProfileReport
prof=ProfileReport(df)
prof.to_file(output_file="rapport_Data_Set.html")

#[1]============ Profilage du jeux de données ================


### Stat sur les cellules vides

In [14]:
df.isnull().sum(axis = 0).sum()

908

In [15]:
df.isnull().sum(axis = 0)

cheveux       103
age            91
exp            96
salaire        95
sexe          100
diplome       110
specialite     93
note          114
dispo         106
embauche        0
dtype: int64

### Rejet des lignes commportant des cellules vides
##### On peut le faire sans perturber le jeu de donnée, les valeurs NaN de chaque colonne représentant 1% du nombre de lignes

In [15]:
df.dropna(inplace=True)

### ne conservons que les rangées comportant des valeurs cohérentes


In [16]:
df = df[df.age > 0]
df = df[df.exp > 0]

## Recherche des corrélations entre variables
##### source méthodologique : https://medium.com/brdata/correlation-straight-to-the-point-e692ab601f4c

#### Construction de 2 dataframes 1 contenant les variables catégorielles et l'autre sans celles-ci

In [19]:
df_objects = df.select_dtypes(include=['object']) # select object type columns only
df_bar_objects = df.select_dtypes(exclude=['object']) # excluding object type column.

##### Vectorization -> Categorical variable converted into countinous variable through integer vector. 

In [20]:
df_objects = df_objects.apply(lambda x : pd.factorize(x)[0]) 
df_final = df_bar_objects.join(df_objects) # Join both dataframes

## pearson : standard correlation coefficient (linear corr)
##### df.corr(method='pearson'). Pearson’s correlation measures the **linear association** between continuous variables.

##### Interpreting the value of ρ :
###### - 0.9 to 1 positive or negative indicates a very strong correlation.
###### - 0.7 to 0.9 positive or negative indicates a strong correlation.
###### - 0.5 to 0.7 positive or negative indicates a moderate correlation.
###### - 0.3 to 0.5 positive or negative indicates a weak correlation.
###### - 0 to 0.3 positive or negative indicates a negligible correlation.


In [23]:
df_filter = df_final.drop(["embauche"], axis=1).copy()
df_filter.corr(method='pearson', min_periods=1)

,age,exp,salaire,note,cheveux,sexe,diplome,specialite,dispo
age,1.000000,-0.005617,-0.000077,-0.004588,0.001421,-0.002809,-0.002091,-0.000544,-0.001165
exp,-0.005617,1.000000,0.009070,-0.011368,0.000800,-0.001680,0.003259,-0.001823,-0.005527
salaire,-0.000077,0.009070,1.000000,-0.450221,0.007885,0.010400,-0.003166,0.008784,0.014764
note,-0.004588,-0.011368,-0.450221,1.000000,0.000547,-0.020450,0.002505,0.016372,0.026477
cheveux,0.001421,0.000800,0.007885,0.000547,1.000000,-0.282810,-0.029611,0.150646,0.124479
sexe,-0.002809,-0.001680,0.010400,-0.020450,-0.282810,1.000000,-0.000399,-0.356891,0.065771
diplome,-0.002091,0.003259,-0.003166,0.002505,-0.029611,-0.000399,1.000000,-0.007399,0.047318
specialite,-0.000544,-0.001823,0.008784,0.016372,0.150646,-0.356891,-0.007399,1.000000,0.366578
dispo,-0.001165,-0.005527,0.014764,0.026477,0.124479,0.065771,0.047318,0.366578,1.000000


##### conclusion il y corrélation (faible) entre : salaire/note, specialité/sexe, spécialité/dispo (négligeable) entre : sexe/cheveux

## Spearman Correlation Coefficient
##### The Spearman correlation coefficient is indicated for the calculation of the correlation between random variables x and y related monotonically to each other, **but not necessarily linearly**. If the relation is linear Pearson’s method is the most indicated.

In [24]:
df_filter = df_final.drop(["embauche"], axis=1).copy()
df_filter.corr(method='spearman', min_periods=1)

,age,exp,salaire,note,cheveux,sexe,diplome,specialite,dispo
age,1.000000,-0.008180,-0.002968,-0.004067,0.001494,-0.002627,-0.002615,-0.000454,-0.000110
exp,-0.008180,1.000000,0.005258,-0.011692,0.001278,-0.002203,0.002896,-0.002094,-0.005321
salaire,-0.002968,0.005258,1.000000,-0.424655,0.007196,0.010903,-0.008189,0.008088,0.015151
note,-0.004067,-0.011692,-0.424655,1.000000,0.000895,-0.021130,0.011842,0.015766,0.026841
cheveux,0.001494,0.001278,0.007196,0.000895,1.000000,-0.290879,0.085806,0.150476,0.125223
sexe,-0.002627,-0.002203,0.010903,-0.021130,-0.290879,1.000000,-0.009727,-0.360798,0.067318
diplome,-0.002615,0.002896,-0.008189,0.011842,0.085806,-0.009727,1.000000,0.009849,0.136684
specialite,-0.000454,-0.002094,0.008088,0.015766,0.150476,-0.360798,0.009849,1.000000,0.372544
dispo,-0.000110,-0.005321,0.015151,0.026841,0.125223,0.067318,0.136684,0.372544,1.000000


##### conclusion il y corrélation (faible) entre : salaire/note, specialité/sexe, spécialité/dispo (négligeable) entre : sexe/cheveux

## Test du Khi-2 (est-ce que des variables catégorielles sont indépendantes ?)

###### Sexe et Spécialité sont 2 variables catégorielles (qui possède un nombre fini de catégories). Pour savoir si ces 2 variables sont liées
###### on utilise le test du Khi-2 [1]. En testant l’hypothèse nulle : “les 2 variables sexe et spécialité sont indépendantes”
###### https://datascientest.com/correlation-entre-variables-comment-mesurer-la-dependance 

#### table de contingence sexe/specialite et test du CHI2

In [33]:
table = pd.crosstab (df["sexe"],df["specialite"])
from scipy.stats import chi2_contingency

In [34]:
resultats_test = chi2_contingency(table)

In [35]:
statistique_de_test = resultats_test[0]
p_valeur = resultats_test[1]
degre_de_liberte = resultats_test[2]

###### Pour décider du rejet ou non de l’hypothèse nulle. On regarde la p_valeur. 
###### Si cette dernière est en-dessous de 5% alors on rejette l’hypothèse nulle.
###### Donc ici le test du Khi-2 nous permet d'affirmer que ces 2 variables sont liées.

In [40]:
if p_valeur <0.05 :  print ("Hypothèse nulle rejetée, les 2 variables sont liées")

Hypothèse nulle rejetée, les 2 variables sont liées


#### Détermination de la force de la relation entre ces 2 variables avec le calcul du V de cramer

In [43]:
import scipy.stats as ss
def cramers_corrected_stat (confusion_matrix):
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr=max(0,phi2-((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

In [44]:
cramers_corrected_stat(table)

0.3695338702710378

##### la relation entre ces 2 variable est modeste, car le calcul du V de cramer corrigé ci-dessus
##### est plus proche de zéro (relation faible) que de 0,9 (relation très forte)

In [49]:
import researchpy as rp

#### Analyse de la variance (ANOVA) à un facteur (corrélation entre une variable continue et une variable catégorielle)

##### si la variable cheveux n'a pas d'influence sur le variable catégorielle Salaire alors leur moyenne devraient être identiques
##### c'est l'hypothèse que nous testons quand on a recours à l'analyse de la variance

In [50]:
#[6] One Way anova with python
#print("\n\n#[6]==========librairie researchpy===============")
#[6.1] Calcul et affiche le calcul de la table Anova du salaire pour toutes les catégories de cheveux
print ("#[6.1] ANOVA ===salaire seul===\n",rp.summary_cont(df['salaire']))
#[6.2] Calcul et affiche le calcul de la table Anova du salaire par couleur de cheveux
print ("#[6.2] ANOVA ===salaire groupé par type de cheveux===\n",
       rp.summary_cont(df['salaire'].groupby(df['cheveux'])))
#[6.3] Calcul et affiche F-statistic and p-value du test
foneway=ss.f_oneway (df['salaire'][df['cheveux']  == 'chatain'],
             df['salaire'][df['cheveux'] == 'brun'],
             df['salaire'][df['cheveux'] == 'blond'],
             df['salaire'][df['cheveux'] == 'roux'])
print ("\n#[6.3]====foneway====\n",foneway)



#[6.1] ANOVA ===salaire seul===
   Variable        N        Mean         SD       SE   95% Conf.    Interval
0  salaire  19905.0  34974.3439  5003.0996  35.4616  34904.8363  35043.8516


#[6.2] ANOVA ===salaire groupé par type de cheveux===
             N        Mean         SD        SE   95% Conf.    Interval
cheveux                                                               
blond    5899  34668.1882  5024.2370   65.4155  34539.9498  34796.4266
brun     5967  35028.5983  4945.5155   64.0226  34903.0908  35154.1058
chatain  5992  35396.5038  5000.6190   64.6008  35269.8630  35523.1447
roux     1944  34432.1039  5034.9706  114.1955  34208.1454  34656.0624

#[6.3]====foneway====
 F_onewayResult(statistic=nan, pvalue=nan)
